# Imports

In [3]:
import subprocess
import os
import pip
import sys
reload(sys)  
sys.setdefaultencoding('utf8')
#os.system("python -m nltk.downloader popular")
def install(package):
    pip.main(['install', package])

In [4]:
try:
    import pandas as pd
except ImportError:
    print 'pandas is not installed, installing it now!'
    install('pandas')
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import issparse, csr_matrix, hstack
sys.path.append('./submissions/starting_kit')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import SparsePCA
from sklearn.preprocessing import normalize, MultiLabelBinarizer
%matplotlib inline

ImportError: No module named matplotlib.pyplot

# Preprocessing

## Loading

In [3]:
train_filename = 'data/train.csv'
train = pd.read_csv(train_filename, sep='\t')
test = pd.read_csv('data/test.csv', sep='\t')
frames = [train, test]
data = pd.concat(frames, axis=0)
data = data.fillna('')
data['date'] = pd.to_datetime(data['date'])
#data['year'] = data.date.dt.year
#data['month'] = data.date.dt.month
#data['day'] = data.date.dt.day
data.drop('date', axis=1, inplace=True)
categorical_columns = ['state','job','source']
replacements = {'state':{'Washington, D.C.':'District of Columbia', 'Tennessee':'Tennesse','Virgina':'Virginia','Washington state':'Washington', 'ohio':'Ohio', 'Rhode island': 'Rhode Island'},'job':{'None':''}}
data = data.replace(to_replace=replacements)
#print(len(np.unique(data.state)))
for col in categorical_columns:
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes

In [5]:
mlb = MultiLabelBinarizer()
df = pd.DataFrame(data[["researched_by",'edited_by']], columns=["researched_by",'edited_by'])
df["edited_by"] = df["edited_by"].str.split(", ")
df["researched_by"] = df["researched_by"].str.split(", ")
data["subjects"] = data["subjects"].str.replace("'", '').str.strip("[]").str.split(", ")
left = df.join(pd.DataFrame(mlb.fit_transform(df.pop('edited_by')),
                          columns=mlb.classes_,
                          index=df.index)).drop(['','researched_by'], axis=1)
right = df.join(pd.DataFrame(mlb.fit_transform(df.pop('researched_by')),
                          columns=mlb.classes_,
                          index=df.index)).drop([''], axis=1)
data =  data.join(pd.DataFrame(mlb.fit_transform(data.pop('subjects')),
                          columns=mlb.classes_,
                          index=data.index))

In [6]:
e = set(left)
r = set(right)
df=right.add(2*left[list(e.intersection(r))]).fillna(right)
data = pd.concat([data.drop(['researched_by','edited_by'],axis=1), df, 2*left[list(e.difference(r))]], axis=1)

(16242, 442)

In [ ]:
%%file submissions/starting_kit/feature_extractor.py
# -*- coding: utf-8 -*-f
from __future__ import unicode_literals

import numpy as np
import string
import unicodeX_df

import sys
reload(sys)  
sys.setdefaultencoding('utf8')

import pandas as pd
from scipy.sparse import issparse, csr_matrix, hstack
sys.path.append('./submissions/starting_kit')


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import SparsePCA
from sklearn.preprocessing import normalize, MultiLabelBinarizer

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler



def clean_str(sentence, stem=True):
    english_stopwords = set(
        [stopword for stopword in stopwords.words('english')])
    punctuation = set(string.punctuation)
    punctuation.update(["``", "`", "..."])
    if stem:
        stemmer = SnowballStemmer('english')
        return list((filter(lambda x: x.lower() not in english_stopwords and
                            x.lower() not in punctuation,
                            [stemmer.stem(t.lower())
                             for t in word_tokenize(sentence)
                             if t.isalpha()])))

    return list((filter(lambda x: x.lower() not in english_stopwords and
                        x.lower() not in punctuation,
                        [t.lower() for t in word_tokenize(sentence)
                         if t.isalpha()])))


def strip_accents_unicode(s):
    try:
        s = unicode(s, 'utf-8')
    except NameError:  # unicode is a default on python 3
        pass
    s = unicodeX_df.normalize('NFD', s)
    s = s.encode('ascii', 'ignore')
    s = s.decode("utf-8")
    return str(s)

from sklearn.feature_extraction.text import TfidfVectorizer
class FeatureExtractor(TfidfVectorizer):
    """Convert a collection of raw documents to a matrix of TF-IDF features. """

    def __init__(self):
        super(FeatureExtractor, self).__init__(
            input='content', encoding='utf-8',
            decode_error='strict', strip_accents=None, lowercase=True,
            preprocessor=None, tokenizer=None, analyzer='word',
            stop_words=None, token_pattern=r"(?u)\b\w\w+\b",
            ngram_range=(1, 1), max_df=1.0, min_df=1,
            max_features=None, vocabulary=None, binary=False,
            dtype=np.int64, norm='l2', use_idf=True, smooth_idf=True,
            sublinear_tf=False)

    def fit(self, X_df, y=None):
        """Learn a vocabulary dictionary of all tokens in the raw documents.

        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.
        Returns
        -------
        self
        """
        X_df = X_df.fillna('')
        X_df['date'] = pd.to_datetime(X_df['date'])
        X_df.drop('date', axis=1, inplace=True)
        categorical_columns = ['state','job','source']
        replacements = {'state':{'Washington, D.C.':'District of Columbia', 'Tennessee':'Tennesse','Virgina':'Virginia','Washington state':'Washington', 'ohio':'Ohio', 'Rhode island': 'Rhode Island'},'job':{'None':''}}
        X_df = X_df.replace(to_replace=replacements)
        for col in categorical_columns:
            X_df[col] = X_df[col].astype('category')
            X_df[col] = X_df[col].cat.codes
        mlb = MultiLabelBinarizer()
        df = pd.X_dfFrame(X_df[["researched_by",'edited_by']], columns=["researched_by",'edited_by'])
        df["edited_by"] = df["edited_by"].str.split(", ")
        df["researched_by"] = df["researched_by"].str.split(", ")
        X_df["subjects"] = X_df["subjects"].str.replace("'", '').str.strip("[]").str.split(", ")
        left = df.join(pd.X_dfFrame(mlb.fit_transform(df.pop('edited_by')),
                                  columns=mlb.classes_,
                                  index=df.index)).drop(['','researched_by'], axis=1)
        right = df.join(pd.X_dfFrame(mlb.fit_transform(df.pop('researched_by')),
                                  columns=mlb.classes_,
                                  index=df.index)).drop([''], axis=1)
        X_df =  X_df.join(pd.X_dfFrame(mlb.fit_transform(X_df.pop('subjects')),
                                  columns=mlb.classes_,
                                  index=X_df.index))
        e = set(left)
        r = set(right)
        df=right.add(2*left[list(e.intersection(r))]).fillna(right)
        X_df = pd.concat([X_df.drop(['researched_by','edited_by'],axis=1), df, 2*left[list(e.difference(r))]], axis=1)
        #y = X_df.pop('truth').values
        #X = hstack((csr_matrix(X_df[X_df.columns[1:]].values), news_sparse))
        self._feat = np.array([' '.join(
            clean_str(strip_accents_unicode(dd)))
            for dd in X_df.statement])
        super(FeatureExtractor, self).fit(self._feat)
        
        return self

    def fit_transform(self, X_df, y=None):
        self.fit(X_df)
        return self.transform(self.X_df)

    def transform(self, X_df):
        X = np.array([' '.join(clean_str(strip_accents_unicode(dd)))
                      for dd in X_df.statement])
        check_is_fitted(self, '_feat', 'The tfidf vector is not fitted')
        X = super(FeatureExtractor, self).transform(X)
        return X

In [7]:
from feature_extractor import FeatureExtractor
fe = FeatureExtractor()
fe.fit(pd.DataFrame(data['statement']))
news_sparse = fe.transform(pd.DataFrame(data['statement']))

In [11]:
data = data.drop(['statement'],axis=1)
y = data.pop('truth').values
X = hstack((csr_matrix(data[data.columns[1:]].values), news_sparse))

In [ ]:
pca = SparsePCA(n_components=200, n_jobs=-1)
X_pca = pca.fit_transform(normalize(csr_matrix(X)).todense())

In [ ]:
X_train, X_test = np.split(X_pca, [train.shape[0]])
y_train, y_test = np.split(y, [train.shape[0]])
#X_train, X_test, y_train, y_test = train_test_split(csr_matrix(X_pca), y, shuffle=False, stratify=None)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [23]:
%%file submissions/starting_kit/classifier.py
# -*- coding: utf-8 -*-
import sys
reload(sys)  
sys.setdefaultencoding('utf8')

from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC


class Classifier(BaseEstimator):
    def __init__(self):
        #self.clf = MLPClassifier()
        clf = LinearSVC(penalty='l2', dual=False,class_weight='balanced')

    def fit(self, X, y):
        self.clf.fit(X.todense(), y)

    def predict(self, X):
        return self.clf.predict(X.todense())

    def predict_proba(self, X):
        return self.clf.predict_proba(X)


In [ ]:
from classifier import Classifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
#clf = LogisticRegression(penalty='l1', fit_intercept=True, class_weight='balanced', solver='saga', multi_class='multinomial', n_jobs=-1)
#clf = MLPClassifier(activation='logistic', solver='lbfgs', hidden_layer_sizes=300)
clf = LinearSVC(penalty='l2', dual=True,class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy_score(y_test,y_pred)

In [ ]:
!ramp_test_submission --quick-test